In [1]:
# one-hot encoding
from konlpy.tag import Komoran
import numpy as np

komoran = Komoran()
text = "오늘 날씨는 구름이 많아요."

# 명사 추출
nouns = komoran.nouns(text)
print(nouns)

# 단어 사전 구축 및 단어별 인덱스 부여
dics = {}

for word in nouns :
    if word not in dics.keys() :
        dics[word] = len(dics)
print(dics)

# one-hot encoding
nb_classes = len(dics)
targets = list(dics.values())
one_hot_targets = np.eye(nb_classes)[targets]
print(one_hot_targets)

['오늘', '날씨', '구름']
{'오늘': 0, '날씨': 1, '구름': 2}
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [3]:
# 희소 표현 / 분산 표현 (= 밀집 표현)
# 희소 표현(sparse representation) : 직관적이지만 메모리 낭비와 계산 복잡도가 커지는 단점이 있다
# 분산 표현(distributed representation) : RGB 와 같이 표현되며 유사성 파악이 가능

In [5]:
# Google Word2Vec Model

from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time

# data read
def read_review_data(filename) :
    with open(filename, 'r') as f :
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]
    return data

# 학습 시간 측정 시작
start = time.time()

# 리뷰 파일 읽어오기
print('1) 말뭉치 데이터 읽기 시작')
review_data = read_review_data('./ratings.txt')
print(len(review_data))     # 리뷰 데이터 전체 개수
print('1) 말뭉치 데이터 읽기 완료 : ', time.time() - start)

# 문장 단위로 명사만 추출해 학습 입력 데이터로
print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran()
docs = [komoran.nouns(sentence[1]) for sentence in review_data]
print('2) 형태소에서 명사만 추출 완료 : ', time.time() - start)

# Word2Vec 모델 학습
print('3) Word2Vec 모델 학습 시작')
model = Word2Vec(sentences=docs, vector_size=200, window=4, hs=1, min_count=2, sg=1)    # size 대신 vector_size 로 해야함
print('3) Word2Vec 모델 학습 완료 : ', time.time() - start)

# 모델 저장
print('4) 학습된 모델 저장 시작')
model.save('nvmc.model')
print('4) 학습된 모델 저장 완료 : ', time.time() - start)

# 학습된 말뭉치 수, 코퍼스 내 전체 단어 수
print("corpus_count : ", model.corpus_count)
print("corpus_total_words : ", model.corpus_total_words)

1) 말뭉치 데이터 읽기 시작
200000
1) 말뭉치 데이터 읽기 완료 :  1.778595209121704
2) 형태소에서 명사만 추출 시작
2) 형태소에서 명사만 추출 완료 :  73.2821798324585
3) Word2Vec 모델 학습 시작
3) Word2Vec 모델 학습 완료 :  83.8989508152008
4) 학습된 모델 저장 시작
4) 학습된 모델 저장 완료 :  84.17003202438354
corpus_count :  200000
corpus_total_words :  1076896


In [6]:
# Word2Vec 모델 활용

# Model Loading
model = Word2Vec.load('nvmc.model')
print("corpus_total_words : ", model.corpus_total_words)

# '사랑'이란 단어로 생성한 단어 임베딩 벡터
print('사랑 : ', model.wv['사랑'])

# 단어 유사도 계산
print("일요일 = 월요일\t", model.wv.similarity(w1='일요일', w2='월요일'))
print("안성기 = 배우\t", model.wv.similarity(w1='안성기', w2='배우'))
print("대기업 = 삼성\t", model.wv.similarity(w1='대기업', w2='삼성'))
print("일요일 != 삼성\t", model.wv.similarity(w1='일요일', w2='삼성'))
print("히어로 != 삼성\t", model.wv.similarity(w1='히어로', w2='삼성'))

# 가장 유사한 단어 추출
print(model.wv.most_similar("안성기", topn=5))
print(model.wv.most_similar("시리즈", topn=5))

corpus_total_words :  1076896
사랑 :  [ 6.31089360e-02 -2.59703219e-01  2.48957574e-01 -2.35638022e-03
 -1.28561199e-01 -3.50652039e-01 -9.62815359e-02  7.97026753e-02
  1.54821798e-01 -9.44212154e-02 -1.63827702e-01  3.87387834e-02
  3.33590396e-02 -6.78327912e-03 -2.89233267e-01  2.02655360e-01
 -5.92268771e-03  4.77764406e-04 -3.07368692e-02 -4.96350855e-01
  1.04046673e-01  1.79063022e-01 -2.15549067e-01  1.54049397e-01
 -3.67549241e-01 -2.40196809e-01  2.67553389e-01 -8.74306560e-02
 -2.73503084e-02 -1.35791570e-01  2.04608575e-01  5.92889972e-02
 -7.76602998e-02 -9.68195871e-02  2.04957008e-01  2.39124268e-01
  1.83949709e-01  1.36689603e-01 -7.34819174e-02  1.74252406e-01
 -3.04551236e-02  2.20781222e-01 -7.63045475e-02 -3.43882650e-01
  4.19012845e-01  3.92550230e-01 -1.10508710e-01 -4.08056043e-02
  4.30987746e-01 -2.50379816e-02  1.86228141e-01 -1.14471480e-01
 -2.09438726e-01 -1.77141372e-02  3.37802678e-01  1.34581238e-01
  1.94393545e-01 -3.15319151e-01  2.06102878e-01  1.85